<a href="https://colab.research.google.com/github/Dr-Ning-An/mcm/blob/main/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
###### Example 5.3 starts here #######

In [40]:
import numpy as np # 导入numpy

In [41]:
# 定义函数由单层板的工程常数计算单层板折减刚度矩阵Q
def transform_engineeringConstants(E1, E2, v12, G12):
    # 根据工程常数计算柔度矩阵的系数
    S11 = 1/E1;
    S12 = -v12/E1;
    S22 = 1/E2;
    S66 = 1/G12;
    # 初始化柔度矩阵
    S = np.zeros((3,3))
    # 将柔度矩阵的非零元素依次赋值；注意Python的编码从0开始。
    S[0][0] = S11;
    S[0][1] = S12;
    S[1][0] = S[0][1];
    S[1][1] = S22;
    S[2][2] = S66;
    # 矩阵运算求解折减刚度矩阵
    Q = np.linalg.inv(S)
    return Q

In [42]:
# 定义函数由单层板折减刚度矩阵Q和铺层角度，计算单层板在全局坐标系下的折减刚度矩阵Q_bar
def transform_Q(Q, theta):
    # 计算转换矩阵 T
    c = np.cos(theta*np.pi/180.0)
    s = np.sin(theta*np.pi/180.0)
    T = np.zeros((3,3))
    T[0][0] = c**2;
    T[0][1] = s**2;
    T[0][2] = 2.0*c*s;
    T[1][0] = s**2;
    T[1][1] = c**2;
    T[1][2] = -2.0*c*s;
    T[2][0] = -s*c;
    T[2][1] = s*c;
    T[2][2] = c**2-s**2;
    # 定义R矩阵
    R = np.zeros((3,3))
    R[0][0] = 1;
    R[1][1] = 1;
    R[2][2] = 2;
    # 计算全局折减刚度矩阵
    Q_bar = np.linalg.multi_dot([np.linalg.inv(T), Q, R, T, np.linalg.inv(R)])
    return Q_bar

In [43]:
def height(thickness):
    # thickness = np.array([5, 5, 5]) # 每一层单层板厚度
    ply_num = len(thickness)
    h_total = np.sum(thickness[0:ply_num]) # 层合板总厚度
    height = np.zeros(ply_num+1)
    #Create array storing the heights of k plies about the midplane
    for k in range(0, ply_num+1):
        height[k]= - h_total/2.0 + np.sum(thickness[0:k])
    return height

In [57]:
def strain(thickness, strain_0, curvature_0): # 已知层合板中性层应变和曲率，求解层合板任意一层的应变分布
    # thickness = np.array([5, 5, 5])
    # strain_0, curvature_0 中性层应变+曲率，6x1
    ply_num = len(thickness)
    height = np.zeros(ply_num+1)
    strain = np.zeros([ply_num+1, 3])
    for k in range(0, ply_num+1):
      strain[k,:] = (strain_0 + height[k]*curvature_0).T
    return strain

In [45]:
def ABD_matrix(Q, statck, thickness):
    A = np.zeros([3, 3])
    B = np.zeros([3, 3])
    D = np.zeros([3, 3])
    h = height(thickness)
    for i in range(3):
        for j in range(3):
            for k in range(len(h)-1):
                Q_bar = transform_Q(Q, statck[k])
                ply_kA = h[k+1] - h[k]
                ply_kB = (h[k+1]**2 - h[k]**2)/2
                ply_kD = (h[k+1]**3 - h[k]**3)/3
                A[i,j] = A[i,j] + ply_kA * Q_bar[i,j]
                B[i,j] = B[i,j] + ply_kB * Q_bar[i,j]
                D[i,j] = D[i,j] + ply_kD * Q_bar[i,j]
    # 将3个3x3的A，B，D矩阵组装称为一个6x6的ABD矩阵
    ABD_matrix = np.concatenate((np.concatenate((A, B), axis = 0), np.concatenate((B, D), axis = 0)), axis = 1)
    return ABD_matrix

In [46]:
# 定义碳环氧单层板已知工程常数
E1 = 181.0*1.0E9; #Pa
E2 = 10.3*1.0E9; #Pa
v12 = 0.28;
G12 = 7.17*1.0E9; #Pa

In [47]:
# 由单层板工程常数计算单层板折减刚度矩阵: Q矩阵
Q = transform_engineeringConstants(E1, E2, v12, G12)
print("折减刚度矩阵：Q =\n", Q, "\n Unit: Pa")

折减刚度矩阵：Q =
 [[1.81811139e+11 2.89692444e+09 0.00000000e+00]
 [2.89692444e+09 1.03461587e+10 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.17000000e+09]] 
 Unit: Pa


In [48]:
# 由单层板折减刚度矩阵Q，以及铺层方式、厚度等求解层合板ABD矩阵
thickness = np.array([0.005, 0.005, 0.005])
statck = np.array([0, 90, 0])
ABD = ABD_matrix(Q, statck, thickness)
print("ABD矩阵：ABD = \n", ABD)

ABD矩阵：ABD = 
 [[ 1.86984218e+09  4.34538667e+07  2.10968854e-09 -9.31322575e-10
   0.00000000e+00  1.07218542e-27]
 [ 4.34538667e+07  1.01251728e+09  5.03863212e-08  0.00000000e+00
   4.07453626e-10  2.56073245e-26]
 [ 2.10968854e-09  5.03863212e-08  1.07550000e+08  1.07218542e-27
   2.56073245e-26  2.91038305e-11]
 [-9.31322575e-10  0.00000000e+00  1.07218542e-27  4.93482893e+04
   8.14760000e+02  4.39518447e-15]
 [ 0.00000000e+00  4.07453626e-10  2.56073245e-26  8.14760000e+02
   4.69595069e+03  1.04971503e-13]
 [ 1.07218542e-27  2.56073245e-26  2.91038305e-11  4.39518447e-15
   1.04971503e-13  2.01656250e+03]]


In [49]:
# 利用层合板宏观力学分析，计算中性面的应变和曲率；
# 按照预设载荷比例，施加单位载荷。
Load = np.array([[1.0], [0.0], [0.0], [0.0], [0.0], [0.0]]) # 题目要求计算单轴拉伸，设置Nx为1，其余项为0
# 中性层的变形为
Deformation = np.linalg.multi_dot([np.linalg.inv(ABD), Load])
# 其中中性层的应变和曲率分别为：
Strain_0 = np.array([Deformation[0], Deformation[1], Deformation[2]])
print("中性层应变：Strain_0 = \n", Strain_0)
Curvature_0 = np.array([Deformation[3], Deformation[4], Deformation[5]])
print("中性层曲率：Curvature_0 = \n", Curvature_0)

中性层应变：Strain_0 = 
 [[ 5.35338415e-10]
 [-2.29749403e-11]
 [ 2.62439807e-28]]
中性层曲率：Curvature_0 = 
 [[ 1.00991586e-23]
 [ 2.41235978e-25]
 [-3.12431998e-41]]


In [81]:
# 计算每一层单层板在全局坐标系下的应变，每层单层板依次计算上中下三个平面的应变
# 第一层 strain_1 6x6
z1 = np.array([-0.075, -0.05, -0.025])
strain_1 = np.zeros([3, 3])
stress_1 = np.zeros([3, 3])
for i in range(3):
  strain_1[i,:] = (Strain_0 + z1[i]*Curvature_0).T
  stress_1[i,:] = (np.linalg.multi_dot([transform_Q(Q, 0.0), strain_1[i,:].T])).T
print("第一层单层板全局坐标系上、中、下表面应变： strain_1 = \n", strain_1)
print("第一层单层板全局坐标系上、中、下表面应力： stress_1 = \n", stress_1)


第一层单层板全局坐标系上、中、下表面应变： strain_1 = 
 [[ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]
 [ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]
 [ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]]
第一层单层板全局坐标系上、中、下表面应力： stress_1 = 
 [[9.72639302e+01 1.31313256e+00 1.88169342e-18]
 [9.72639302e+01 1.31313256e+00 1.88169342e-18]
 [9.72639302e+01 1.31313256e+00 1.88169342e-18]]


In [82]:
# 第二层 strain_2 6x6
z2 = np.array([-0.025, 0.0, 0.025])
strain_2 = np.zeros([3, 3])
stress_2 = np.zeros([3, 3])
for i in range(3):
  strain_2[i,:] = (Strain_0 + z2[i]*Curvature_0).T
  stress_2[i,:] = (np.linalg.multi_dot([transform_Q(Q, 90.0), strain_2[i,:].T])).T
print("第二层单层板全局坐标系上、中、下表面应变： strain_2 = \n", strain_2)
print("第二层单层板全局坐标系上、中、下表面应力： stress_2 = \n", stress_2)

第二层单层板全局坐标系上、中、下表面应变： strain_2 = 
 [[ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]
 [ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]
 [ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]]
第一层单层板全局坐标系上、中、下表面应力： stress_2 = 
 [[ 5.47213955e+00 -2.62626512e+00 -3.76338683e-18]
 [ 5.47213955e+00 -2.62626512e+00 -3.76338683e-18]
 [ 5.47213955e+00 -2.62626512e+00 -3.76338683e-18]]


In [83]:
# 第三层 strain_3 6x6
z3 = np.array([-0.025, 0.0, 0.025])
strain_3 = np.zeros([3, 3])
stress_3 = np.zeros([3, 3])
for i in range(3):
  strain_3[i,:] = (Strain_0 + z2[i]*Curvature_0).T
  stress_3[i,:] = (np.linalg.multi_dot([transform_Q(Q, 0.0), strain_3[i,:].T])).T
print("第三层单层板全局坐标系上、中、下表面应变： strain_3 = \n", strain_3)
print("第三层单层板全局坐标系上、中、下表面应力： stress_3 = \n", stress_3)

第三层单层板全局坐标系上、中、下表面应变： strain_3 = 
 [[ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]
 [ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]
 [ 5.35338415e-10 -2.29749403e-11  2.62439807e-28]]
第三层单层板全局坐标系上、中、下表面应力： stress_3 = 
 [[9.72639302e+01 1.31313256e+00 1.88169342e-18]
 [9.72639302e+01 1.31313256e+00 1.88169342e-18]
 [9.72639302e+01 1.31313256e+00 1.88169342e-18]]
